# 範例目標:

運用 Numpy 讀取或輸出不同檔案格式


# 範例重點:

1. 注意不同檔案格式有不同的讀取、輸出方式
2. .npy 與 .npz 格式是NumPy的檔案格式，透過 save()、savez()、load() 函式進行儲存與讀取
3. 針對文字檔，可以使用 savetxt()、loadtxt()、genfromtxt() 來儲存與讀取

# [教學目標]

* 了解並且學會使用陣列中的統計方法
* 了解並且學會使用陣列中的搜尋與排序方法
* 了解並且學會使用陣列中的形狀方法


In [33]:
import numpy as np

## 1. `numpy.save()`、`numpy.savez()`、`numpy.load()`

`numpy.save()` 是將**單一陣列儲存到 .npy 格式的函式，**

而 `numpy.savez()` 可以將**多個陣列儲存到同一個 .npz 格式的檔案中。**

讀取 .npy / .npz 檔案，使用 `numpy.load()` 函式來**開啟檔案，並回傳檔案中的陣列。**

呼叫 `numpy.save()` 時，儲存多個陣列時，內容會依序附加 (append) 在該**檔案的最後。**

使用 `numpy.savez()` 時，可以儲存多個陣列。

當呼叫 `numpy.load()` 載入 .npz 檔案時，回傳的會是 **NpzFile 類別。**

透過 `files` 屬性回傳的 List，可以看到載入的物件

相較於 CSV 或 TXT 檔案，開啟 NumPy 格式的檔案在效能上快非常多。

![](https://miro.medium.com/max/984/1*xwpjjSdZwiOMnPJtdp9L2w.png)

來源網址：[URL](https://towardsdatascience.com/what-is-npy-files-and-why-you-should-use-them-603373c78883)

儲存單一陣列到 .npy 檔案，並用 `numpy.load()` 載入回傳陣列。

In [34]:
with open('one_array.npy', 'wb') as f:
    np.save(f, np.array([1, 2]))

In [35]:
np.load('one_array.npy')

array([1, 2])

**`numpy.save()` 只能存單一陣列**

In [36]:
with open('test.npy', 'wb') as f:
    np.save(f, np.array([1, 2]))
    np.save(f, np.array([1, 3]))
    np.save(f, np.array([1, 4]))
    np.save(f, np.array([1, 3]))

載入的時候每一次 `numpy.load()` 就載入一個陣列。

In [37]:
with open('test.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)
    c = np.load(f)
    d = np.load(f)

print(a, b, c, d)

[1 2] [1 3] [1 4] [1 3]


**`numpy.savez()` 時，可以儲存多個陣列**。下面範例在儲存陣列時並指定陣列關鍵字 (array1, array2...)，若未指定的話預設會以 arr_0, arr_1... 關鍵字設定。

In [38]:
x = np.arange(10)
y = np.array([1, 2, 3])
z = np.random.rand(10)

with open('multi_array.npz', 'wb') as f:
    np.savez(f, array1=x, array2=y, array3=z)

當呼叫 `numpy.load()` 載入 .npz 檔案時，回傳的會是 NpzFile 類別。

In [39]:
npzfile = np.load('multi_array.npz')
type(npzfile)

numpy.lib.npyio.NpzFile

透過 files 屬性回傳的 List，可以看到載入的物件裡面包含 3 個陣列，名稱分別為 array1, array2, array3

In [40]:
npzfile.files

['array1', 'array2', 'array3']

**顯示每一個陣列的內容。**

In [41]:
print(npzfile['array1'])
print(npzfile['array2'])
print(npzfile['array3'])

[0 1 2 3 4 5 6 7 8 9]
[1 2 3]
[0.82271518 0.5898398  0.61418199 0.03843925 0.06787261 0.66983922
 0.63174277 0.99515424 0.40559107 0.86778909]


# 2. `savetxt()` 與 `loadtxt()`

### 2.1 `numpy.savetxt()`

`savetxt()` 可將一維或是二維陣列儲存到文字檔，<br>
並且可以設定元素值的格式、分隔符號、換行字元、檔頭 (header)、檔尾 (footer)、檔案字元編碼... 等引數。

函式的用法如下：

```python
numpy.savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='n', header='', footer='', comments='# ', encoding=None)
```

   使用 `fmt` 引數可以指定輸出的格式

   在存檔時也可以加入 header / footer 做為檔案註解說明。

```
ex: np.savetxt('test.out', x, fmt='%1.4e', delimiter=',', header='this is,\nheader', footer='this is footer')
```

引數的定義如下表：其中僅 fname 是必輸欄位。

|引數名稱|定義|預設值|
|---|---|---|
|fname|檔案名稱||
|X|要儲存的一維或二維陣列||
|fmt|陣列元素的格式，例如科學記號的格式定義(%1.4e)、整數(%d)、浮點數(%f)...|%.18e|
|delimiter|分隔符號|空格|
|newline|換行字元|n|
|header|檔頭註解文字|空字串|
|footer|檔尾註解文字|空字串|
|comments|註解文字的前綴字元或字串|#加一空格|
|encoding|檔案的字元編碼|`None`|<br>
<br>

**使用 `%load <filename>` magic command 來查看檔案內容。**

副檔名為 .gz : 存檔時會存為壓縮的 gzip 檔案。<br>

- 一維陣列<br>

  **需注意，如果儲存的陣列是一維的話，須加上中括號才能正常產生符號分隔檔格式，否則分隔符號會被忽略。**<br>

  
  

- 二維陣列則沒有上述的情況。<br>





In [42]:
x = y = z = np.arange(0.0,5.0,1.0)
x

array([0., 1., 2., 3., 4.])

In [43]:
y = np.arange(10).reshape(2, 5)
y

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

**需注意，如果儲存的陣列是一維的話，須加上中括號才能正常產生符號分隔檔格式，否則分隔符號會被忽略。**

In [44]:
np.savetxt('test.out', [x], delimiter=',')

使用 `%load <filename>` magic command 來查看檔案內容。

In [ ]:
# 此magic command會出現下面
%load test.out      

In [ ]:
%load test.out
0.000000000000000000e+00,1.000000000000000000e+00,2.000000000000000000e+00,3.000000000000000000e+00,4.000000000000000000e+00


In [ ]:
np.savetxt('test.gz', [x], delimiter=',')

In [ ]:
np.savetxt('test.csv', y, delimiter=',')

In [ ]:
#此magic command會出現下面
%load test.csv       

In [ ]:
# %load test.csv
0.000000000000000000e+00,1.000000000000000000e+00,2.000000000000000000e+00,3.000000000000000000e+00,4.000000000000000000e+00
5.000000000000000000e+00,6.000000000000000000e+00,7.000000000000000000e+00,8.000000000000000000e+00,9.000000000000000000e+00


使用 `fmt` 引數可以指定輸出的格式，下例是指定科學記號的格式來輸出陣列值。

在存檔時也可以加入 header / footer 做為檔案註解說明。

In [ ]:
np.savetxt('test.out', x, fmt='%1.4e', delimiter=',', header='this is,\nheader', footer='this is footer')

In [ ]:
#此magic command會出現下面
%load test.out                     

In [ ]:
# %load test.out
# this is,
# header
0.0000e+00
1.0000e+00
2.0000e+00
3.0000e+00
4.0000e+00
# this is footer


### 2.2 `numpy.loadtxt()`

函式的用法如下：

```python
numpy.loadtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, converters=None, skiprows=0, usecols=None, unpack=False, ndmin=0, encoding='bytes', max_rows=None)
```

引數的定義如下表：其中僅 fname 是必輸欄位。

|引數名稱|定義|預設值|
|---|---|---|
|fname|檔案名稱||
|dtype|陣列的資料型別|float|
|comments|註解文字的前綴字元或字串|#|
|delimiter|分隔符號|None|
|converters|以字典型別定義 {column:轉換函式} key/value值|None|
|skiprows|讀取時要略過開頭的row數目(例如註解的行數)|0|
|usecols|要讀取的column|None|
|unpack|bool值，如果是True的話，會轉置(transpose)輸出的陣列|False|
|ndmin|設定傳回陣列的最低軸數|0|
|encoding|檔案的字元編碼|bytes|
|max_rows|在skiprows的row數目後，最大的讀取row數目，預設讀取所有資料|None|

**如果檔案是 gz 或是 bz2 壓縮檔的話，可以直接讀取不需要先解壓縮。**

`loadtxt()` 函式與 `genfromtxt()` 函式有一些相同的引數及功能，但是 `genfromtxt()` 功能更有彈性<br><br>

讀取儲存成文字檔的陣列時，使用 `loadtxt()` 載入，

下面例子是載入我們上面儲存的 `test.out` 文字檔。

這邊可以看到載入寺預設的資料型別是 `float`，而原先儲存時使用的科學記號格式，在載入時被轉換為浮點數格式。

呼叫時可用 `delimiter` 引數指定分隔符號來正確載入陣列資料。

在 `dtype` 引數中 `f4` 代表的是浮點數 4 bytes，也就是 `float32`。

In [ ]:
np.loadtxt('test.out', delimiter=',', dtype='f4')

## 3. `genfromtxt()`

跟 `loadtxt()` 相比，`genfromtxt()` 提供更 powerful 及更有彈性的功能，用來讀取文字檔格式的陣列。

函式用法如下：

```python
numpy.genfromtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, skip_header=0, skip_footer=0, converters=None, missing_values=None, filling_values=None, usecols=None, names=None, excludelist=None, deletechars=" !#$%&'()*+, -./:;<=>?@[\]^{|}~", replace_space='_', autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, usemask=False, loose=True, invalid_raise=True, max_rows=None, encoding='bytes')
```

引數的定義如下表：其中僅 fname 是必輸欄位。

|引數名稱|定義|預設值|
|---|---|---|
|fname|檔案名稱或是輸入資料||
|dtype|陣列的資料型別|float|
|comments|註解文字的前綴字元或字串|#|
|delimiter|分隔符號|None|
|skip_header|讀取時忽略的檔頭行數|0|
|skip_footer|讀取時忽略的檔尾行數|0|
|converters|以字典型別定義 {column:轉換函式} key/value值|None|
|missing_values|用來識別缺值的字串|None|
|filling_values|用來填入缺值的值|None|
|usecols|要讀取的column|None|
|names|column名稱|None|
|excludelist|要排除的column名稱|None|
|deletechars|須從column名稱中刪除的字元|#$%&'()*+, -./:;<=>?@[\]^{|}~|
|replace_space|要取代空格的字元|_|
|autostrip|是否自動去空格|False|
|case_sensitive|欄位名稱是否區分大小寫，可以設定True/False/upper(轉為大寫)/lower(轉為小寫)|True|
|defaultfmt|如果names未定義完整名稱，defaultfmt可用來定義structured dtype的column名稱|f%i|
|unpack|bool值，如果是True的話，會轉置(transpose)輸出的陣列|False|
|usemask|如果是True的話，回傳masked array；否則回傳正常的array|False|
|loose|如果是True的話，無效的值不會導致錯誤|True|
|invalid_raise|設為True時，如果column數目不合會拋出exception；如果設為False的話，拋出warning並且跳過不合數目的資料|True|
|max_rows|在skiprows的row數目後，最大的讀取row數目，預設讀取所有資料|None|
|encoding|檔案的字元編碼|bytes|
                 
**跟 `loadtxt()` 相同，如果檔案是 gz 或是 bz2 壓縮檔的話，可以直接讀取不需要先解壓縮。**
                 


### 3.1 將文字檔內容讀取並正確分隔Column

要將文字檔內容讀入並正確分隔Column，才能獲得預期中的陣列及元素值。

常用的分隔符號有逗號、tab... 

下面的範例中分別示範了逗號分隔以及固定寛度的元素值，要如何讀取。

最基本的用法就是讀取CSV檔案。預設的分隔符號為None，所以在這邊我們必須指定正確的分隔符號。

除了單獨讀取文字檔，字串 List 也可以做為輸入。
                 
類檔案的物件都可以做為輸入，例如 `StringIO`。<br>

  - 當 `delimiter` 給定的是一個整數、或是整數的序列時，可以用來將固定寬度的字串讀入。**固定寬度包含了空格。**
                 
  - 如果給定的是單一整數代表所有陣列元素都是同一寬度；如有不同寬度時，可以使用整數序列來定義。
                 
`autostrip` 引數如果設為 `True`，在讀取時會自動將元素值的空格去除。
                 
在檔案內容中包含了以 comments 啟始的 header / footer。
                 
設定要略過的行數就可以正確讀入欲讀取的陣列元素。

與 `loadtxt()` 相同，讀取時可以略過註解文字，或是 header / footer。

在檔案內容中包含了以 # 啟始的 header / footer。header 有 2 行而 footer 有 1 行，設定要略過的行數就可以正確讀入欲讀取的陣列元素。

    ex: %load test.out
        # this is,
        # header
        0.0000e+00
        1.0000e+00
        2.0000e+00
        3.0000e+00
        4.0000e+00
        # this is footer
     
   `np.genfromtxt("test.out", comments=None, skip_footer=1, skip_header=2)`

In [ ]:
np.genfromtxt("test.csv", delimiter=",")

跟 `loadtxt()` 相同，如果檔案是 gz 或是 bz2 壓縮檔的話，可以直接讀取不需要先解壓縮。

In [ ]:
np.genfromtxt("test.gz", delimiter=",")

字串 List 也可以做為輸入。

In [ ]:
np.genfromtxt(["1", "2", "abc", "4", "5"])

類檔案的物件都可以做為輸入，例如 `StringIO`。

當 `delimiter` 給定的是一個整數、或是整數的序列時，可以用來將固定寬度的字串讀入，

在下面的範例中，**固定寬度包含了空格。**

如果給定的是單一整數代表所有陣列元素都是同一寬度；如有不同寬度時，可以使用整數序列來定義。

In [53]:
from io import StringIO

data = u"  1  2  3\n  4  5 67\n890123  4"
np.genfromtxt(StringIO(data), delimiter=3)

array([[  1.,   2.,   3.],
       [  4.,   5.,  67.],
       [890., 123.,   4.]])

In [54]:
data = u"123456789\n   4  7 9\n   4567 9"
np.genfromtxt(StringIO(data), delimiter=(4, 3, 2))

array([[1234.,  567.,   89.],
       [   4.,    7.,    9.],
       [   4.,  567.,    9.]])

`autostrip` 引數如果設為 `True`，在讀取時會自動將元素值的空格去除。

In [55]:
data = u"1, 2 , 4\n 4, 5, 6"
np.genfromtxt(StringIO(data), delimiter=",", autostrip=True)

array([[1., 2., 4.],
       [4., 5., 6.]])

與 `loadtxt()` 相同，讀取時可以略過註解文字，或是 header / footer。

In [56]:
np.genfromtxt("test.out", comments="#")

array(nan)

在檔案內容中包含了以 # 啟始的 header / footer。header 有 2 行而 footer 有 1 行，設定要略過的行數就可以正確讀入欲讀取的陣列元素。

In [57]:
np.genfromtxt("test.out", comments=None, skip_footer=1, skip_header=2)

D:\Anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:1: UserWarning: genfromtxt: Empty input file: "test.out"
  """Entry point for launching an IPython kernel.


array([], dtype=float64)

### 3.2 選擇要讀取的 Column

`names` 引數是用來指明是否檔案內容中有Column名稱，

或是如果原來內容沒有的話，可以給定Column名稱。

`names=True` 代表這個讀入的內容中有Column名稱，也就是這個檔案中的第一行。若有指定 `skip_header` 的話則會是 header 後的第一行。

若是原始資料中沒有名稱，可以透過 `names` 指定。

透過 `usecols` 引數可以選擇要讀入的Column

如果沒有Column名稱的話，可以使用整數指定要讀取的Column索引值。

**但是若已有 `names` 的話，使用索引值會產生錯誤訊息。**

如果沒有給定 `names` 或是給的數目少於Column，那麼在回傳結構化陣列時，會自動以 `f%i` 的命名規則產生 `names`。

若要指定命名規則，可以使用 `defaultfmt` 引數。

P.S. <br>
   >int8, int16, int32, int64 四種數據類型可以使用字符串 'i1', 'i2','i4','i8' 代替<br>
   字節順序是通過對數據類型預先設定 < 或 > 來決定的。 <br>
   < 意味著小端法(最小值存儲在最小的地址，即低位組放在最前面)。<br> 
   \> 意味著大端法(最重要的字節存儲在最小的地址，即高位組放在最前面)。<br>
   


In [ ]:
#此magic command會出現下面
%load names.txt

In [ ]:
# %load names.txt
a,b,c
1,2,3
4,5,6
7,8,9

`names=True` 代表這個讀入的內容中有Column名稱，也就是這個檔案中的第一行。若有指定 `skip_header` 的話則會是 header 後的第一行。

In [58]:
np.genfromtxt("names.txt", delimiter=",", names=True)

array([(1., 2., 3.), (4., 5., 6.), (7., 8., 9.)],
      dtype=[('a', '<f8'), ('b', '<f8'), ('c', '<f8')])

若是原始資料中沒有名稱，可以透過 `names` 指定。

In [63]:
data = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(data, names="a, b, c")

array([(1., 2., 3.), (4., 5., 6.)],
      dtype=[('a', '<f8'), ('b', '<f8'), ('c', '<f8')])

透過 `usecols` 引數可以選擇要讀入的Column，下面的例子是指定要讀入的Column名稱。

In [64]:
a = u"1,2,3,4,5\n6,7,8,9,10"
np.genfromtxt(StringIO(a), delimiter=",", names="a, b, c", usecols=("a", "c"))

array([(1., 3.), (6., 8.)], dtype=[('a', '<f8'), ('c', '<f8')])

如果沒有Column名稱的話，可以使用整數指定要讀取的Column索引值。

In [65]:
a = u"1 2 3 4 5\n6 7 8 9 10"
np.genfromtxt(StringIO(a), usecols=(1, -1))

array([[ 2.,  5.],
       [ 7., 10.]])

但是若已有 `names` 的話，使用索引值會產生錯誤訊息。

In [66]:
a = u"1 2 3 4 5\n6 7 8 9 10"
np.genfromtxt(StringIO(a), names="a, b, c", usecols=(1, -1))

IndexError: list index out of range

如果沒有給定 `names` 或是給的數目少於Column，那麼在回傳結構化陣列時，會自動以 `f%i` 的命名規則產生 `names`。

In [67]:
a = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(a, dtype=(int, float, int))

array([(1, 2., 3), (4, 5., 6)],
      dtype=[('f0', '<i4'), ('f1', '<f8'), ('f2', '<i4')])

若要指定命名規則，可以使用 `defaultfmt` 引數。

In [68]:
a = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(a, dtype=(int, float, int), defaultfmt="var_%i")

array([(1, 2., 3), (4, 5., 6)],
      dtype=[('var_0', '<i4'), ('var_1', '<f8'), ('var_2', '<i4')])

### 3.3 缺值處理

預設空值都被視為缺值 (missing value)，用 `filling_values` 可以指定要填值 (filling value)。

In [69]:
a = u", 2, 3\n4, ,"
np.genfromtxt(StringIO(a), delimiter=",", filling_values=np.nan)

array([[nan,  2.,  3.],
       [ 4., nan, nan]])

除了空值之外，若有**特定字串應被視為缺值的話，使用 `missing_values` 引數可以指定，而且可以使用序列來指定缺值與填值。**

要留意的是，使用字串序列的話，要每個Column依序指定。

In [70]:
a = u"N/A, 2, 3, ???"
np.genfromtxt(StringIO(a), delimiter=",", 
              missing_values=["N/A", "N/A", "N/A", "???"], 
              filling_values=[0, 0, 0, -999])

array([   0.,    2.,    3., -999.])

針對不同的 `dtype`，如果沒有指定填值的話，根據不同的型別的缺值有不同的預設填值。

|dtype|預設填值|
|---|---|
|bool|False|
|int|-1|
|float|np.nan|
|complex|np.nan+0j|
|string|''|

In [71]:
a = u"1, , \n , 5, 6"
np.genfromtxt(StringIO(a), delimiter=',', dtype="int, float, str")

array([( 1, nan, ''), (-1,  5., '')],
      dtype=[('f0', '<i4'), ('f1', '<f8'), ('f2', '<U')])

### 3.4 資料轉換

在讀取檔案時使用 `converters` 引數可以同時轉換資料。在範例檔案中，資料包含Yes/No與百分比，將在讀取時進行轉換。

P.S.

>在字串的前面加上 b or u : <br>
「b」表示「bytes」，表示內容是bytes，而非字串（str）<br>
「u」的意思是：unicode，它是從python 2就存在的，為了讓這個字串可以有準確的編碼。但在python 3的預設文字編碼方式就是unicode，所以在python 3可以不用特別設定這個，如果不放心還是可加上u

In [73]:
np.genfromtxt("transform.txt", delimiter=',', dtype="i8, i8, U3, U3")

array([(1, 2, 'Yes', '87%'), (3, 4, 'No', '3%'), (5, 6, 'Yes', '55%')],
      dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<U3'), ('f3', '<U3')])

舉例來說，如果我們想將資料中的Yes/No與百分比進行轉換，使用自訂函式來進行。

In [78]:
def trans(s):
    if s == b'Yes':
        return 1
    else:
        return 0

In [79]:
def conversion(x):
    return float(x.strip(b"%"))/100

`converters` 引數接收的是字典型別 (dictionary)，key 代表的是Column，可以使用索引或是names定義的Column名稱。下面的例子是使用索引。

In [80]:
np.genfromtxt("transform.txt", delimiter=',', converters={2:trans, 3:conversion})

array([(1., 2., 1, 0.87), (3., 4., 0, 0.03), (5., 6., 1, 0.55)],
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<i4'), ('f3', '<f8')])

# NumPy 運算

**陣列中的統計方法 :**

`.sum()`<br>
`.min()`<br>
`.max()`<br>

In [81]:
# 陣列中的統計方法


a = np.arange(6)
print(a)

print(a.sum()) 
print(a.min()) 
print(a.max()) 


[0 1 2 3 4 5]
15
0
5


**一種功能，三種函式**

In [82]:
# 一種功能，三種函式


a = np.arange(6)

print(a.sum()) 
print(np.sum(a))
print(sum(a)) 

15
15
15


**陣列中的軸參數**

In [83]:
# 陣列中的軸參數

b = np.arange(12).reshape(3, 4)

print(b)
print(b.sum())
print(b.sum(axis=0))
print(b.sum(axis=1))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
66
[12 15 18 21]
[ 6 22 38]


**搜尋與排序方法**

In [84]:
# 搜尋與排序方法

import numpy as np 

a = np.array([1, 3, 2, 5, 4])
print(a.sort())
print(a)

None
[1 2 3 4 5]


`np.sort`

In [85]:
import numpy as np 

a = np.array([1, 3, 2, 5, 4])
print(np.sort(a))
print(a)

[1 2 3 4 5]
[1 3 2 5 4]


`np.searchsorted`

In [86]:
import numpy as np 

print(np.searchsorted([1,2,3,4,5], 3))
print(np.searchsorted(
    [1, 2, 3, 4, 5],
    [-10, 10, 3, 5]
))


2
[0 5 2 4]


**reshape 和 resize**

In [87]:
# reshape 和 resize

import numpy as np 

a = np.arange(6)

print(a)
print(a.reshape(3, 2))
print(a)


[0 1 2 3 4 5]
[[0 1]
 [2 3]
 [4 5]]
[0 1 2 3 4 5]


In [88]:
import numpy as np 

a = np.arange(6)

print(a)
print(a.resize((3, 2)))
print(a)


[0 1 2 3 4 5]
None
[[0 1]
 [2 3]
 [4 5]]


**三種陣列攤平用法**

In [89]:
# 三種陣列攤平用法

import numpy as np 

a = np.arange(6).reshape((3, 2))

print(a.ravel() )
print(a.flatten())
print(a.flat)
print(list(a.flat))

[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0, 1, 2, 3, 4, 5]
